In [1]:
import numpy as np
import pandas as pd 
import warnings
import pickle
import wandb
import time
import tqdm

In [19]:
api = wandb.Api(timeout=60)
entity, project = 'the-real-dl', 'bio-transformers'
runs = api.runs(entity + '/' + project)
dataset = 'housing'

In [3]:
your_sweep_ids = [
    'sa2rga9s', # housing-bp-adagrad
    'kr6c1nli', # housing-bp-adam
    'vpmuxl79', # housing-bp-rmsprop
    'wqly0d6f', # housing-bp-momentum
    'jvmkpt8s', # housing-pc-adagrad-adam
    'jdcczics', # housing-pc-adam-adam
    'u1otip1g', # housing-pc-rmsprop-adam
    'njukee5z', # housing-pc-momentum-adam
    'ev5y7qxc', # housing-pc-adagrad-momentum
    'wosduku0', # housing-pc-adam-momentum
    'y71cxyo5', # housing-pc-rmsprop-momentum
    'm47oc6ou', # housing-pc-momentum-momentum
]

In [4]:
def extract(wandb_runs, maxrun=None):
    
    dfs = []
    count = 0

    for i, run in enumerate(runs):
        # time.sleep(2) # to avoid 429 Client Error
        print(f'Fetching run #{i}', end='\r')
        if maxrun is not None and count == maxrun: break

        summary = run.summary
        if not 'epoch' in summary.keys(): continue
        if run.state == 'running': continue
        if run.state == 'crashed': continue
        if run.state == 'failed': continue

        run_id = run.id
        sweep_id = run.sweep.id

        if sweep_id not in your_sweep_ids: continue

        config = run.config
        run_df = run.history()

        config.update({'sweep-id': sweep_id, 'run-id': run_id})
        summary.update({'sweep-id': sweep_id, 'run-id': run_id})

        l_epochs = []
        l_train_loss = []
        l_test_loss = []
        l_train_energy = []

        for e in range(summary['epoch']):
            x = run_df[run_df.epoch == e]
            if len(x.index) == 0: continue

            train_losses = x.train_loss.unique()
            test_losses = x.test_loss.unique()

            train_loss_no_nan = train_losses[~np.isnan(train_losses)]
            test_loss_no_nan = test_losses[~np.isnan(test_losses)]

            train_loss = train_loss_no_nan[0] if len(train_loss_no_nan) > 0 else np.nan
            test_loss = test_loss_no_nan[0] if len(test_loss_no_nan) > 0 else np.nan

            l_epochs.append(e)
            l_train_loss.append(train_loss)
            l_test_loss.append(test_loss)

            if config['training'] == 'pc':
                train_energies = x.train_energy.unique()
                train_energy_no_nan = train_energies[~np.isnan(train_energies)]
                train_energy = train_energy_no_nan[0] if len(train_energy_no_nan) > 0 else np.nan
                l_train_energy.append(train_energy)

            if config['training'] == 'bp':
                run_ids = np.full(len(l_epochs), run_id)
                sweep_ids = np.full(len(l_epochs), sweep_id)
                data = list(zip(sweep_ids, run_ids, l_epochs, l_train_loss, l_test_loss))
                columns = ['sweep_id', 'run_id', 'epoch', 'train_loss', 'test_loss']
            else:
                run_ids = np.full(len(l_epochs), run_id)
                sweep_ids = np.full(len(l_epochs), sweep_id)
                data = list(zip(sweep_ids, run_ids, l_epochs, l_train_loss, l_test_loss, l_train_energy))
                columns = ['sweep_id', 'run_id', 'epoch', 'train_loss', 'test_loss', 'train_energy']

        df = pd.DataFrame(data, columns=columns)

        # adding configs to df
        for k in list(config.keys())[:-2]:
            df['c_' + str(k)] = config[k]

        # adding summary to df
        for k in summary.keys():
            if k in ['_runtime', 'train_loss', 'test_loss', 'train_energy']:
                df['f_' + str(k)] = summary[k]

        dfs.append(df)
        count += 1


    print(f'Fetched {count} runs in {runs.entity}/{runs.project} (max runs: {"all" if maxrun is None else maxrun})')
    return pd.concat(dfs, axis=0).reset_index(drop=True)

In [5]:
def sort_runs_by_test_loss(df, top=None):
    x = df.drop(columns=['epoch', 'train_loss', 'test_loss', 'train_energy'])
    x = x.drop(columns=['c_loss', 'c_seed', 'c_model', 'c_wandb', 'c_device'])
    x = x.drop(columns=['c_epochs', 'c_dataset', 'c_dropout', 'c_batch-size'])

    if x.c_training.unique()[0] == 'bp':
        x = x.drop(columns=[c for c in list(x.columns) if c.startswith('c_pc')])
        x = x.drop(columns=['c_clr', 'c_init', 'c_iterations', 'c_x_optimizer', 'f_train_energy'])

    x = x.drop_duplicates().reset_index(drop=True)
    x = x.sort_values('f_test_loss', ascending=True).reset_index(drop=True)

    return x if top is None else x[:top]

In [6]:
def top_parameters(df):
    df_params = pd.DataFrame(columns=['parameter', 'abs_best', 'mean_best', 'median_best'])
    params = [c for c in df.columns if c.startswith('c')]

    for param in params:
        abs_best = df.loc[0, param]
        mean_best = df.groupby(param).agg({'f_test_loss': np.mean}).reset_index().iloc[0, 0]
        median_best = df.groupby(param).agg({'f_test_loss': np.median}).reset_index().iloc[0, 0]
        df_params = df_params.append({'parameter': param, 'abs_best': abs_best, 'mean_best': mean_best, 'median_best': median_best}, ignore_index=True)

    return df_params

In [7]:
def correlation_stats(df):
    corr_train_loss = df.corr()['train_loss']
    corr_test_loss = df.corr()['test_loss']
    corr_train_energy = df.corr()['train_energy']
    corr_train_loss = corr_train_loss[corr_train_loss.index.str.startswith(('c_'))].sort_values(ascending=False)
    corr_test_loss = corr_test_loss[corr_test_loss.index.str.startswith(('c_'))].sort_values(ascending=False)
    corr_train_energy = corr_train_energy[corr_train_energy.index.str.startswith(('c_'))].sort_values(ascending=False)
    return corr_train_loss, corr_test_loss, corr_train_energy

In [8]:
def plot_run(df, run_id):
    xlabel = 'epochs'
    ylabel = df[df.run_id == run_id].c_loss.unique()[0]
    title = f'run {run_id}'
    # we can add fancy std-err visualizations and co.
    df[df.run_id == run_id].train_loss.plot(x='epoch', legend='train loss', xlabel=xlabel, ylabel=ylabel, title=title, figsize=(10, 4))
    df[df.run_id == run_id].test_loss.plot(x='epoch', legend='test loss', xlabel=xlabel, ylabel=ylabel, title=title, figsize=(10, 4))

In [9]:
# with warnings.catch_warnings():
df = extract(runs, maxrun=None)
df.to_csv(f'{dataset}-wandb-runs.csv')

df_bp = df[(df.c_training == 'bp')]
df_pc = df[(df.c_training == 'pc')]

Fetched 2398 runs in the-real-dl/bio-transformers (max runs: all)


In [10]:
top_bp = sort_runs_by_test_loss(df_bp, top=100)
top_pc = sort_runs_by_test_loss(df_pc, top=100)

In [11]:
top_bp[:5]

,sweep_id,run_id,c_lr,c_training,c_optimizer,c_batch_size,c_weight_decay,c_learning_rate,f_test_loss,f_train_loss,f__runtime,c_gamma,c_momentum
0,wqly0d6f,5z77r722,0.005,bp,momentum,1,0.0,0.005,0.355353,0.000238,857.349233,0.5,0.9
1,wqly0d6f,cl2kipww,0.005,bp,momentum,1,0.0,0.005,0.355353,0.000238,1928.212445,0.5,0.9
2,wqly0d6f,xl6nj9ui,0.005,bp,momentum,1,0.0,0.005,0.355353,0.000238,2673.520079,0.5,0.9
3,wqly0d6f,q41g7tc9,0.005,bp,momentum,1,0.0,0.005,0.355353,0.000238,2531.568782,0.5,0.9
4,wqly0d6f,8q2yu03t,0.005,bp,momentum,1,0.0,0.005,0.355353,0.000238,553.719509,0.5,0.9


In [12]:
top_pc[:5]

,sweep_id,run_id,c_lr,c_training,c_optimizer,c_batch_size,c_weight_decay,c_learning_rate,f_test_loss,f_train_loss,...,c_momentum,c_pc_gamma,c_pc_iters,c_iterations,c_pc_momentum,c_x_optimizer,c_pc_optimizer,c_pc_weight_dacay,c_pc_weight_decay,f_train_energy
0,jdcczics,wqoxpsze,0.01,pc,adam,32,0.0,0.01,0.367675,0.086584,...,NaN,NaN,50.0,50.0,NaN,adam,adam,0.001,0.001,0.114549
1,jdcczics,c47o9m0k,0.01,pc,adam,32,0.0,0.01,0.367675,0.086584,...,NaN,NaN,50.0,50.0,NaN,adam,adam,0.001,0.001,0.114549
2,jdcczics,fjgyt2ac,0.01,pc,adam,32,0.0,0.01,0.367675,0.086584,...,NaN,NaN,50.0,50.0,NaN,adam,adam,0.001,0.001,0.114549
3,jdcczics,ilkltr95,0.01,pc,adam,32,0.0,0.01,0.367675,0.086584,...,NaN,NaN,50.0,50.0,NaN,adam,adam,0.001,0.001,0.114549
4,jdcczics,pvfc1epr,0.01,pc,adam,32,0.0,0.01,0.367675,0.086584,...,NaN,NaN,50.0,50.0,NaN,adam,adam,0.001,0.001,0.114549


In [13]:
top_parameters(top_bp)

,parameter,abs_best,mean_best,median_best
0,c_lr,0.005,0.0001,0.0001
1,c_training,bp,bp,bp
2,c_optimizer,momentum,adam,adam
3,c_batch_size,1,1,1
4,c_weight_decay,0,0,0
5,c_learning_rate,0.005,0.0001,0.0001
6,c_gamma,0.5,0,0
7,c_momentum,0.9,0.3,0.3


In [14]:
top_parameters(top_pc)

,parameter,abs_best,mean_best,median_best
0,c_lr,0.01,0.0001,0.0001
1,c_training,pc,pc,pc
2,c_optimizer,adam,adagrad,adagrad
3,c_batch_size,32,1,1
4,c_weight_decay,0,0,0
5,c_learning_rate,0.01,0.0001,0.0001
6,c_clr,0.1,0.001,0.001
7,c_init,forward,forward,forward
8,c_gamma,NaN,0,0
9,c_pc_clr,0.1,0.001,0.001


In [15]:
_, bp_corr_test_loss, _ = correlation_stats(df_bp)
_, pc_corr_test_loss, _ = correlation_stats(df_pc)

In [16]:
bp_corr_test_loss

c_momentum           0.006151
c_weight_decay       0.005779
c_batch_size         0.003295
c_batch-size         0.003295
c_learning_rate      0.002713
c_lr                 0.002713
c_gamma             -0.000322
c_seed                    NaN
c_epochs                  NaN
c_dropout                 NaN
c_clr                     NaN
c_pc_clr                  NaN
c_pc_gamma                NaN
c_pc_iters                NaN
c_iterations              NaN
c_pc_momentum             NaN
c_pc_weight_dacay         NaN
c_pc_weight_decay         NaN
Name: test_loss, dtype: float64

In [17]:
pc_corr_test_loss

c_momentum           0.024217
c_pc_clr             0.013107
c_clr                0.013107
c_pc_gamma           0.008457
c_pc_momentum        0.003968
c_learning_rate      0.001593
c_lr                 0.001593
c_epochs            -0.002571
c_gamma             -0.007024
c_pc_weight_decay   -0.012247
c_pc_weight_dacay   -0.012247
c_batch_size        -0.015771
c_batch-size        -0.015771
c_weight_decay      -0.016544
c_seed                    NaN
c_dropout                 NaN
c_pc_iters                NaN
c_iterations              NaN
Name: test_loss, dtype: float64